# Anwendungsbeispiel:  Texte mit Python analysieren

## Erkennung der Sprache anhand der Häufigkeit von Buchstaben und n-Grammen

Wir wollen uns heute mit der statistischen Auswertung von Texten beschäftigen und Versuchen, die Zugehörigkeit eines Textes zu einer Sprache anhand der Verteilung von Buchstaben und *n-Grammen* zu bestimmen.

In [175]:
def ngrams(word, n):
    """ Generiere alle n-Gramme eines Wortes """
    if len(word) < n:
        yield word
    for index in range(len(word) - (n - 1)): 
        yield word[index:index+n]

In [145]:
[ g for g in ngrams('Py', 3) ]

['Py']

In [184]:
class Distribution:
    """ Generiere eine Statistik für Buchstaben oder n-Gramme """
    
    def __init__(self):
        self.counter = dict()
        self.count = 0
        
    def add(self, iter):
        """ Füge Elemente aus iter hinzu """
        for x in iter:
            self.count += 1
            if x in self.counter:
                self.counter[x] += 1
            else:
                self.counter[x] = 1
    
    def normalize(self):
        """ Gebe normalisierte Verteilung zurück """
        return { k:c/self.count for (k, c) in self.counter.items() }


In [147]:
import time

def read_dict(name):
    """ Lese das Wörterbuch `name` und erstelle eine Statistik der Buchstabenhäufigkeit.
        Das Wörterbuch sollte je Zeile ein Wort enthalten und in UTF-8 kodiert sein.
    """
    letters = Distribution()
    bigrams = Distribution()
    words = 0

    start = time.time()
    with open(name, 'r') as file:
        for word in file:
            word = word[:-1].lower()
            words += 1
            
            letters.add(word)
            bigrams.add(ngrams(word, 2))
    end = time.time()

    print(f"read {words:,d} words [{letters.count:,d} chars, {bigrams.count:,d} bigrams] from {name} in {end - start:.1f} seconds")
    return (letters.normalize(), bigrams.normalize())


In [148]:
GERMAN = read_dict('german.txt')
ENGLISH = read_dict('english.txt')
SPANISH = read_dict('español.txt')

read 1,996,021 words [29,056,458 chars, 27,060,437 bigrams] from german.txt in 14.6 seconds
read 370,103 words [3,494,694 chars, 3,124,617 bigrams] from english.txt in 1.8 seconds
read 191,667 words [1,936,738 chars, 1,745,074 bigrams] from español.txt in 1.0 seconds


In [149]:
import operator

for c in sorted(GERMAN[0].items(), key=operator.itemgetter(1), reverse=True)[:10]:
    print(f"{c[0]} => Deutsch: {100*GERMAN[0][c[0]]:5.2f} %  English: {100*ENGLISH[0][c[0]]:5.2f} %  Spanisch: {100*SPANISH[0][c[0]]:5.2f} %")

e => Deutsch: 15.19 %  English: 10.77 %  Spanisch: 10.03 %
n => Deutsch:  9.15 %  English:  7.19 %  Spanisch:  6.30 %
s => Deutsch:  7.93 %  English:  7.16 %  Spanisch:  5.03 %
r => Deutsch:  7.85 %  English:  7.04 %  Spanisch:  7.84 %
t => Deutsch:  7.05 %  English:  6.61 %  Spanisch:  4.22 %
i => Deutsch:  6.04 %  English:  8.96 %  Spanisch:  5.89 %
a => Deutsch:  5.79 %  English:  8.46 %  Spanisch: 14.05 %
l => Deutsch:  4.57 %  English:  5.58 %  Spanisch:  7.36 %
h => Deutsch:  4.15 %  English:  2.64 %  Spanisch:  0.63 %
u => Deutsch:  3.99 %  English:  3.76 %  Spanisch:  2.33 %


In [150]:
for c in sorted(GERMAN[1].items(), key=operator.itemgetter(1), reverse=True)[:10]:
    print(f"{c[0]} => Deutsch: {100*GERMAN[1][c[0]]:5.2f} %  English: {100*ENGLISH[1][c[0]]:5.2f} %  Spanisch: {100*SPANISH[1][c[0]]:5.2f} %")

en => Deutsch:  4.00 %  English:  1.23 %  Spanisch:  1.53 %
er => Deutsch:  3.68 %  English:  2.14 %  Spanisch:  1.18 %
ch => Deutsch:  2.58 %  English:  0.63 %  Spanisch:  0.33 %
te => Deutsch:  2.42 %  English:  1.42 %  Spanisch:  0.86 %
ge => Deutsch:  1.76 %  English:  0.38 %  Spanisch:  0.11 %
st => Deutsch:  1.65 %  English:  1.09 %  Spanisch:  0.85 %
ei => Deutsch:  1.53 %  English:  0.14 %  Spanisch:  0.06 %
ng => Deutsch:  1.45 %  English:  0.95 %  Spanisch:  0.23 %
un => Deutsch:  1.44 %  English:  0.88 %  Spanisch:  0.19 %
es => Deutsch:  1.31 %  English:  1.50 %  Spanisch:  1.76 %


In [180]:
import math
import re

def euclid(a, b):
    """ Berechne den euklidischen Abstand zweier normalisierten Verteilungen """
    sum = 0

    # Erst die Schnittmenge
    for x in a.keys() & b.keys():
        sum += (a[x] - b[x])**2

    # Dann die Differenzmengen
    for x in a.keys() - b.keys():
        sum += a[x] * a[x]
    for x in b.keys() - a.keys():
        sum += b[x] * b[x]

    return math.sqrt(sum)
   
def hellinger(a, b):
    """ Berechne die Hellinger-Distanz zweier normalisierter Verteilungen """
    sum = 0

    # Erst die Schnittmenge
    for x in a.keys() & b.keys():
        sum += 0.5 * (math.sqrt(a[x]) - math.sqrt(b[x]))**2

    # Dann die Differenzmengen
    for x in a.keys() - b.keys():
        sum += 0.5 * a[x]
    for x in b.keys() - a.keys():
        sum += 0.5 * b[x]

    return math.sqrt(sum)


def analyze(text):
    """ Analysiere Text und berechne den Abstand zu den Sprachen """
    letters = Distribution()
    bigrams = Distribution()
    
    text = text.lower()
    for word in re.sub("[^\w]+", " ", text).split():
        letters.add(word)
        bigrams.add(ngrams(word, 2))
            
     
    delta = dict()
    for (lang, dist) in [ ('Deutsch', GERMAN), ('Englisch', ENGLISH), ('Spanisch', SPANISH) ]:
        delta[lang] = ( euclid(letters.normalize(), dist[0]), hellinger(letters.normalize(), dist[0]),
                        euclid(bigrams.normalize(), dist[1]), hellinger(bigrams.normalize(), dist[1]) ) 
        
    return delta

def print_report(delta):
    """ Gebe Resultate aus """
    ranking = { lang : val[3] for (lang, val) in delta.items() }
    ranking = sorted(ranking.items(), key=operator.itemgetter(1))
    
    print(f"Der Text ist wahrscheinlich {ranking[0][0]}")
    print()
    print("Buchstaben:")
    for res in ranking:
        lang = res[0]
        print(f"    Sprache: {lang:8} Euklidischer Abstand: {delta[lang][0]:.2f}  Hellinger-Abstand: {delta[lang][1]:.2f}")

    print("Bigramme:")
    for res in ranking:
        lang = res[0]
        print(f"    Sprache: {lang:8} Euklidischer Abstand: {delta[lang][2]:.2f}  Hellinger-Abstand: {delta[lang][3]:.2f}")
        
        

In [181]:
# "Stufen", Hermann Hesse

text = """
Wie jede Blüte welkt und jede Jugend
Dem Alter weicht, blüht jede Lebensstufe,
Blüht jede Weisheit auch und jede Tugend
Zu ihrer Zeit und darf nicht ewig dauern.

Es muß das Herz bei jedem Lebensrufe
Bereit zum Abschied sein und Neubeginne,
Um sich in Tapferkeit und ohne Trauern
In andre, neue Bindungen zu geben.

Und jedem Anfang wohnt ein Zauber inne,
Der uns beschützt und der uns hilft, zu leben.

Wir sollen heiter Raum um Raum durchschreiten,
An keinem wie an einer Heimat hängen,
Der Weltgeist will nicht fesseln uns und engen,
Er will uns Stuf' um Stufe heben, weiten.

Kaum sind wir heimisch einem Lebenskreise
Und traulich eingewohnt, so droht Erschlaffen,
Nur wer bereit zu Aufbruch ist und Reise,
Mag lähmender Gewöhnung sich entraffen.

Es wird vielleicht auch noch die Todesstunde
Uns neuen Räumen jung entgegensenden,
Des Lebens Ruf an uns wird niemals enden ...
Wohlan denn, Herz, nimm Abschied und gesunde!
"""

delta = analyze(text)
print_report(delta)


Der Text ist wahrscheinlich Deutsch

Buchstaben:
    Sprache: Deutsch  Euklidischer Abstand: 0.09  Hellinger-Abstand: 0.17
    Sprache: Englisch Euklidischer Abstand: 0.14  Hellinger-Abstand: 0.27
    Sprache: Spanisch Euklidischer Abstand: 0.19  Hellinger-Abstand: 0.36
Bigramme:
    Sprache: Deutsch  Euklidischer Abstand: 0.07  Hellinger-Abstand: 0.47
    Sprache: Englisch Euklidischer Abstand: 0.11  Hellinger-Abstand: 0.60
    Sprache: Spanisch Euklidischer Abstand: 0.12  Hellinger-Abstand: 0.68


In [182]:
# "The road not taken", Robert Frost

text = """ 
Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth;

Then took the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted wear;
Though as for that the passing there
Had worn them really about the same,

And both that morning equally lay
In leaves no step had trodden black.
Oh, I kept the first for another day!
Yet knowing how way leads on to way,
I doubted if I should ever come back.

I shall be telling this with a sigh
Somewhere ages and ages hence:
Two roads diverged in a wood, and I—
I took the one less traveled by,
And that has made all the difference.
"""

delta = analyze(text)
print_report(delta)


Der Text ist wahrscheinlich Englisch

Buchstaben:
    Sprache: Englisch Euklidischer Abstand: 0.09  Hellinger-Abstand: 0.18
    Sprache: Deutsch  Euklidischer Abstand: 0.11  Hellinger-Abstand: 0.23
    Sprache: Spanisch Euklidischer Abstand: 0.11  Hellinger-Abstand: 0.30
Bigramme:
    Sprache: Englisch Euklidischer Abstand: 0.09  Hellinger-Abstand: 0.52
    Sprache: Deutsch  Euklidischer Abstand: 0.10  Hellinger-Abstand: 0.60
    Sprache: Spanisch Euklidischer Abstand: 0.11  Hellinger-Abstand: 0.62


In [183]:
# "Cantares", Antonio Machado

text = """ 
Todo pasa y todo queda, 
pero lo nuestro es pasar, 
pasar haciendo caminos, 
caminos sobre el mar. 

Nunca perseguí la gloria, 
ni dejar en la memoria 
de los hombres mi canción; 
yo amo los mundos sutiles, 
ingrávidos y gentiles, 
como pompas de jabón. 

Me gusta verlos pintarse 
de sol y grana, volar 
bajo el cielo azul, temblar 
súbitamente y quebrarse... 

Nunca perseguí la gloria. 

Caminante, son tus huellas 
el camino y nada más; 
caminante, no hay camino, 
se hace camino al andar. 

Al andar se hace camino 
y al volver la vista atrás 
se ve la senda que nunca 
se ha de volver a pisar. 

Caminante no hay camino 
sino estelas en la mar... 

Hace algún tiempo en ese lugar 
donde hoy los bosques se visten de espinos 
se oyó la voz de un poeta gritar 
"Caminante no hay camino, 
se hace camino al andar..." 

Golpe a golpe, verso a verso... 

Murió el poeta lejos del hogar. 
Le cubre el polvo de un país vecino. 
Al alejarse le vieron llorar. 
"Caminante no hay camino, 
se hace camino al andar..." 

Golpe a golpe, verso a verso... 

Cuando el jilguero no puede cantar. 
Cuando el poeta es un peregrino, 
cuando de nada nos sirve rezar. 
"Caminante no hay camino, 
se hace camino al andar..." 

Golpe a golpe, verso a verso.
"""

delta = analyze(text)
print_report(delta)


Der Text ist wahrscheinlich Spanisch

Buchstaben:
    Sprache: Spanisch Euklidischer Abstand: 0.06  Hellinger-Abstand: 0.18
    Sprache: Englisch Euklidischer Abstand: 0.09  Hellinger-Abstand: 0.19
    Sprache: Deutsch  Euklidischer Abstand: 0.13  Hellinger-Abstand: 0.29
Bigramme:
    Sprache: Spanisch Euklidischer Abstand: 0.09  Hellinger-Abstand: 0.50
    Sprache: Englisch Euklidischer Abstand: 0.09  Hellinger-Abstand: 0.54
    Sprache: Deutsch  Euklidischer Abstand: 0.11  Hellinger-Abstand: 0.63
